## Importing libraries

In [1]:
import pandas as pd
import cbsodata
import json
import re
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# Assuming file paths are generated for CSV files
file_paths = [os.path.join(DATA_DIR, f"{identifier}.csv") for identifier in identifier_freq_dict.keys()]

def load_csv_files(file_paths):
    """
    Load CSV files into DataFrames.
    """
    dataframes = []
    for file_path in file_paths:
        try:
            print(f"Loading file: {file_path}")
            df = pd.read_csv(file_path)
            dataframes.append(df)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    return dataframes

# Load tables from CSV files
tables = load_csv_files(file_paths)

# Process the loaded tables
if tables:  # Check if there are any tables loaded
    final_df = process_tables(tables, identifier_freq_dict)

    # Save the final combined DataFrame
    final_output_path = os.path.join(DATA_DIR, 'merged_tables.csv')
    final_df.to_csv(final_output_path, index=False)
    print(f"Final DataFrame saved to {final_output_path}.")
else:
    print("No tables were loaded. Cannot proceed with processing.")


Loading file: C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\83451NED.csv
Loading file: C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\85928NED.csv
Loading file: C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\80072ned.csv
Loading file: C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\81589NED.csv
Loading file: C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\83147NED.csv
Loading file: C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\81588NED.csv
Loading file: C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\83149NED.csv
Loading file: C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\83148NED.csv
Final DataFrame saved to C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\mer